# 과제#1. 더반찬 주문량 예측

## 0. 환경준비

### 0.1 Import

In [26]:
# 기본 라이브러리 가져오기
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split  

### 0.2 Data Loading

In [45]:
# raw data 로딩 
raw_data = pd.read_csv(r'C:\Users\user\Desktop\intern.csv')
raw_data.head()

,Unnamed: 0,ORD_NO,H_YMD,O_YMD,PKG_GOODS_NO,PKG_GOODS_NM,GOODS_NO,GOODS_NM,STD_GSGR_NO_LEV1_NM,ORD_QTY,CANCEL_QTY,RET_QTY,REAL_ORD_QTY,RECVR_ROAD_BASE_ADDR,SALE_PRICE,DISCOUNT_AMT,FINAL_PRICE
0,0,202201095519105,2022-01-11 00:00:00,2022-01-09 12:26:10,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,충청북도 증평군 증평읍 송산로 11 (지평더웰아파트),6900,569.0,6331
1,1,202201105522898,2022-01-11 00:00:00,2022-01-10 10:35:49,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,서울특별시 강남구 논현로85길 52 (역삼동),6900,177.0,6723
2,2,202201115527172,2022-01-12 00:00:00,2022-01-11 11:38:32,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,"서울특별시 송파구 올림픽로4길 42 (잠실동, 우성아파트)",6900,221.0,6679
3,3,202201105523467,2022-01-12 00:00:00,2022-01-10 13:21:51,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,강원도 속초시 도리원길 11-14 (노학동),6900,128.0,6772
4,4,202201105522519,2022-01-11 00:00:00,2022-01-10 08:16:21,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,"경기도 용인시 수지구 동천로 64 (동천동, 동천마을동문굿모닝힐5차아파트)",6900,157.0,6743


In [52]:
# 카피 데이터 생성
copy_data = raw_data.copy()

# 희망배송일 순서대로 데이터를 정렬
copy_data = copy_data.sort_values(by = 'H_YMD')


copy_data.head()

,Unnamed: 0,ORD_NO,H_YMD,O_YMD,PKG_GOODS_NO,PKG_GOODS_NM,GOODS_NO,GOODS_NM,STD_GSGR_NO_LEV1_NM,ORD_QTY,CANCEL_QTY,RET_QTY,REAL_ORD_QTY,RECVR_ROAD_BASE_ADDR,SALE_PRICE,DISCOUNT_AMT,FINAL_PRICE
166288,119348,201912302355791,2020-01-01 00:00:00,2019-12-30 21:59:40,단품,단품,14375,옛날잡채(500g),메인요리,1,0,0,1,"경기도 수원시 장안구 경수대로976번길 22 (조원동, 수원 한일타운)",8600,1408.0,7192
37753,37753,201912302355557,2020-01-01 00:00:00,2019-12-30 20:50:07,단품,단품,1901012353,수제계란말이(350g),반찬,2,0,0,2,서울 성북구 동소문로34길 24 삼성아파트,6300,NaN,6300
162585,115645,201912302353567,2020-01-01 00:00:00,2019-12-30 11:02:39,단품,단품,1823,숙주나물(300g),반찬,1,0,0,1,"서울특별시 마포구 삼개로 33 (도화동, 도화3지구우성아파트)",3300,486.0,2814
164041,117101,201912302354611,2020-01-01 00:00:00,2019-12-30 15:19:58,단품,단품,1712010310,두메산나물비빔밥재료,메인요리,1,0,0,1,경기도 용인시 수지구 문인로3번길 22 (풍덕천동),6900,264.0,6636
164040,117100,201912302352965,2020-01-01 00:00:00,2019-12-30 10:08:58,단품,단품,1712010310,두메산나물비빔밥재료,메인요리,1,0,0,1,"경기도 구리시 아차산로487번길 27 (교문동, 아차산어울림아파트)",6900,317.0,6583


In [53]:
# 타겟이 되는 20개 제품을 설정한다
target = ['GOODS_NM__심방골주부X더반찬_ 시골 돼지짜글이(600g)',
       'GOODS_NM_가정집 오징어불고기/셀프(380g)', 'GOODS_NM_건고사리나물볶음(150g)',
       'GOODS_NM_건표고버섯볶음', 'GOODS_NM_고구마 품은 라자냐(450g)',
       'GOODS_NM_고소한도토리묵무침(360g)', 'GOODS_NM_꼬막무침 (260g)',
       'GOODS_NM_두메산나물비빔밥재료', 'GOODS_NM_메밀소바(2인분)', 'GOODS_NM_셀프두부조림(600g)',
       'GOODS_NM_소고기유니짜장소스(1인분, 200g)', 'GOODS_NM_수제계란말이(350g)',
       'GOODS_NM_숙주나물(300g)', 'GOODS_NM_순살코다리강정(180g)', 'GOODS_NM_양장피',
       'GOODS_NM_열무비빔밥재료믹스(2인분)', 'GOODS_NM_옛날잡채(500g)',
       'GOODS_NM_우삼겹숙주볶음(250g)', 'GOODS_NM_채소계란찜(340g)',
       'GOODS_NM_한돈 제육볶음(700g)']

In [54]:
# 만들어진 데이터 기준 121행부터 2022년 5월의 데이터이므로 2022년 5월 이후 데이터를 test 데이터로 사용한다
# 104: 2022 전체 데이터
# 121: 2022.05 데이터

# train_data = copy_data[:104]
# test_data = copy_data[104:]

train_data = copy_data[:121]
test_data = copy_data[121:]

## 1. 데이터 전처리

### 1.0 결측치 확인

In [24]:
# 결측치가 있는 데이터 확인
train_data.isna().sum()

Unnamed: 0               0
ORD_NO                   0
H_YMD                    0
O_YMD                    0
PKG_GOODS_NO             0
PKG_GOODS_NM             0
GOODS_NO                 0
GOODS_NM                 0
STD_GSGR_NO_LEV1_NM      0
ORD_QTY                  0
CANCEL_QTY               0
RET_QTY                  0
REAL_ORD_QTY             0
RECVR_ROAD_BASE_ADDR     0
SALE_PRICE               0
DISCOUNT_AMT            42
FINAL_PRICE              0
dtype: int64

### 1.1 예측 대상과 다른 데이터를 제거

In [9]:
def DBC_DATA_REMOVE(df):
    # 예측 대상에는 없는 상품들은 모두 nan값으로 변경
    df['GOODS_NM'].loc[df['GOODS_NM'] == '꼬막무침 (250g)'] = np.nan
    df['GOODS_NM'].loc[df['GOODS_NM'] == '고소한도토리묵무침(265g)'] = np.nan
    df['GOODS_NM'].loc[df['GOODS_NM'] == '고사리나물볶음(150g)'] = np.nan

    return df

### 1.2 잘못 적재되어 있는 데이터 전처리

In [10]:
def DBC_SET_OR_SINGLE(df):
    # 내일한정, 한정판매는 단품으로 대체
    # 한정판매 중에는 단품이 아닌 세트도 있어서 이를 유의해야 함(ex. '[한정판매]손님초대상 set', '[한정판매]반찬 한 상 set', '[한정판매]키즈 set')

    # 내일한정이 붙은 제품은 모두 단품인 것을 확인하여 카테고리를 단품으로 바꿔줌, 
    # [내일한정] 고소한도토리묵무침(265g), [내일한정] 건고사리나물볶음(150g), [내일한정] 꼬막무침 (250g)은 제외
    df['PKG_GOODS_NM'].loc[df['PKG_GOODS_NM'].str.contains('내일한정')] = '단품'

    # 한정판매 중에서 두 제품만 단품이었기에 두 제품을 단품으로 바꿔줌
    df['PKG_GOODS_NM'].loc[df['PKG_GOODS_NM'] == '[한정판매] 두메산나물비빔밥재료'] = '단품'  
    df['PKG_GOODS_NM'].loc[df['PKG_GOODS_NM'] == '[한정판매] 옛날잡채(500g)'] = '단품'

    # 단품으로 바꿔주지 않은 모든 제품을 세트로 바꿔줌
    df['PKG_GOODS_NM'].loc[df['PKG_GOODS_NM'] != '단품'] = '세트'

    return df

### 1.3 데이터 타입 변환

In [36]:
def DBC_CHANGE_TYPE(df):
    # 자재코드를 추후 가변수화를 위해 문자열 변수로 변환
    df['GOODS_NO'] = df['GOODS_NO'].astype(str)

    # 희망배송일 변수를 datetime 형식으로 변환
    df['H_YMD'] = pd.to_datetime(df['H_YMD'])

    df.head()

    return df

### 1.4 특정 컬럼의 결측치 채우기 

In [12]:
def DBC_FILL_NAN(df):
    # 할인액의 NaN값을 모두 0으로 대체
    # 할인되지 않은 상품을 산것으로 추측되어 이 경우 할인액은 0이 되는게 맞다.
    df['DISCOUNT_AMT'].fillna(0, inplace=True)

    # 위에서 만든 예측 대상과 다른 데이터에 들어있는 NaN 값을 모두 제거
    df.dropna(inplace=True)

    return df

### 1.5 Feature Engineerung

In [33]:
def DBC_FEATURE_ENGINEERING(df):
    
    # 할인율을 나타내는 SALE_PERCENTAGE 변수 생성
    # 할인액 / 판매가격 * 100
    df['SALE_PERCETANGE'] = (df['DISCOUNT_AMT'] / df['SALE_PRICE']) * 100

    # 희망배송일의 연도,월,일,시간,분,초를 각각 컬럼으로 만들어줌
    df['year'] = df['H_YMD'].dt.year
    df['month'] = df['H_YMD'].dt.month
    df['day'] = df['H_YMD'].dt.day
    df['hour'] = df['H_YMD'].dt.hour
    df['minute'] = df['H_YMD'].dt.minute
    df['second'] = df['H_YMD'].dt.second

    return df

### 1.6 컬럼 제거

In [14]:
def REMOVE_COLUMNS(df):
    # 불필요한 컬럼들 제거
    # Unnamed: 단순 데이터 순서를 나타내는 컬럼이기에 삭제
    # O_YMD: 주문 시간은 오더 건수와 큰 상관이 없기에 삭제
    # ORD_NO: 주문 내용의 일련번호이기에 삭제
    # PKG_GOODS_NO: PKG_GOODS_NM과 내용이 겹치고 잘못 적재된 데이터이기에 삭제
    # RECVR_ROAD_BASE_ADDR: 비대면으로 주문하는 시대에 지역정보는 큰 의미가 없을 것 같아 삭제
    dummy_col = ['Unnamed: 0', 'O_YMD', 'ORD_NO', 'PKG_GOODS_NO', 'RECVR_ROAD_BASE_ADDR']
    df = df.drop(dummy_col, axis=1)

    return df


### 1.7 이상치 제거

In [15]:
def DBC_OUTLIER(df):
    ol_col = ['ORD_QTY', 'CANCEL_QTY', 'RET_QTY', 'REAL_ORD_QTY', 'SALE_PRICE', 'FINAL_PRICE']
    
    for col in ol_col: 
        q1 = df[col].quantile(0.25) # 1사분위
        q3 = df[col].quantile(0.75) # 3사분위 
        iqr = q3 - q1               # iqr 수치

        df.loc[(df[col] > (q3 + 1.5 * iqr)) | (df[col] < (q1 - 1.5 * iqr)), col] = df[col].median() # 이상치를 중앙값으로 대체
    
    return df


### 1.8 가변수화

In [66]:
def DBC_GET_DUMMIES(df):
    cols = ['PKG_GOODS_NM', 'GOODS_NO', 'GOODS_NM', 'STD_GSGR_NO_LEV1_NM']
    df= pd.get_dummies(df, columns = cols)

    # 데이터의 컬럼에 '['나 ','같은 특수기호가 들어가 있으면 학습이 안되기 때문에 제거해준다.
    import re
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df.columns.values]
    
    return df

### 1.9 데이터를 타겟에 맞게 변형

In [17]:
def DBC_RESAMPLING(df):
    # 데이터를 주 단위로 groupby 하고 해당 주에 팔린 상품의 개수를 sum()한다.
    resampled = df.resample(rule='1W', on='H_YMD')   
    resampled.sum()
    df = resampled.sum()

    return df

## 2. 데이터 분할

### 2.0 컬럼명 전처리

In [ ]:
y = train_data[target]
X = train_data.drop(target, axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .3, random_state = 2022)

In [18]:
def DBC_TARGET(df):
   # 데이터의 컬럼에 '['나 ','같은 특수기호가 들어가 있으면 학습이 안되기 때문에 제거해준다.
   import re
   regex = re.compile(r"\[|\]|<", re.IGNORECASE)
   df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df.columns.values]

   # 타겟이 되는 20개 제품을 설정한다
   target = ['GOODS_NM__심방골주부X더반찬_ 시골 돼지짜글이(600g)',
       'GOODS_NM_가정집 오징어불고기/셀프(380g)', 'GOODS_NM_건고사리나물볶음(150g)',
       'GOODS_NM_건표고버섯볶음', 'GOODS_NM_고구마 품은 라자냐(450g)',
       'GOODS_NM_고소한도토리묵무침(360g)', 'GOODS_NM_꼬막무침 (260g)',
       'GOODS_NM_두메산나물비빔밥재료', 'GOODS_NM_메밀소바(2인분)', 'GOODS_NM_셀프두부조림(600g)',
       'GOODS_NM_소고기유니짜장소스(1인분, 200g)', 'GOODS_NM_수제계란말이(350g)',
       'GOODS_NM_숙주나물(300g)', 'GOODS_NM_순살코다리강정(180g)', 'GOODS_NM_양장피',
       'GOODS_NM_열무비빔밥재료믹스(2인분)', 'GOODS_NM_옛날잡채(500g)',
       'GOODS_NM_우삼겹숙주볶음(250g)', 'GOODS_NM_채소계란찜(340g)',
       'GOODS_NM_한돈 제육볶음(700g)']
   
   y = df[target]
   X = df.drop(target, axis=1)

   X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .3, random_state = 2022)
   
   return X_train, X_val, y_train, y_val

In [30]:
# # 범위가 제각각인 각 컬럼들을 0~1 사이에 범위로 조정해준다.

# from sklearn.preprocessing import MinMaxScaler
# col_x = list(X)
# scaler = MinMaxScaler()
# X_train_s = scaler.fit_transform(X_train)
# X_val_s = scaler.transform(X_val)
# X_test_s = scaler.transform(X_test)

In [69]:
def DBC_DATAPIPELINE(df, scaler):
    temp = df.copy()

    temp = DBC_DATA_REMOVE(temp)

    temp = DBC_SET_OR_SINGLE(temp)

    temp = DBC_CHANGE_TYPE(temp)

    temp = DBC_FILL_NAN(temp)

    temp = DBC_FEATURE_ENGINEERING(temp)

    temp = REMOVE_COLUMNS(temp)

    temp = DBC_OUTLIER(temp)

    temp = DBC_GET_DUMMIES(temp)

    temp = DBC_RESAMPLING(temp)

   
    # 타겟이 되는 20개 제품을 설정한다
    target = ['GOODS_NM__심방골주부X더반찬_ 시골 돼지짜글이(600g)',
       'GOODS_NM_가정집 오징어불고기/셀프(380g)', 'GOODS_NM_건고사리나물볶음(150g)',
       'GOODS_NM_건표고버섯볶음', 'GOODS_NM_고구마 품은 라자냐(450g)',
       'GOODS_NM_고소한도토리묵무침(360g)', 'GOODS_NM_꼬막무침 (260g)',
       'GOODS_NM_두메산나물비빔밥재료', 'GOODS_NM_메밀소바(2인분)', 'GOODS_NM_셀프두부조림(600g)',
       'GOODS_NM_소고기유니짜장소스(1인분, 200g)', 'GOODS_NM_수제계란말이(350g)',
       'GOODS_NM_숙주나물(300g)', 'GOODS_NM_순살코다리강정(180g)', 'GOODS_NM_양장피',
       'GOODS_NM_열무비빔밥재료믹스(2인분)', 'GOODS_NM_옛날잡채(500g)',
       'GOODS_NM_우삼겹숙주볶음(250g)', 'GOODS_NM_채소계란찜(340g)',
       'GOODS_NM_한돈 제육볶음(700g)']
    temp = temp.drop(target, axis=1)
    x_cols = list(temp)
    temp = scaler.transform(temp)

    return pd.DataFrame(temp, columns=x_cols)


In [70]:
scaler = joblib.load('C:\\Users\\user\\Dongwon\\Dongwon_Project\\더반찬_XGB모델\\dbc_scaler.pkl')
DBC_DATAPIPELINE(test_data, scaler)

C:\Users\user\AppData\Local\Temp\ipykernel_4060\3931068277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GOODS_NM'].loc[df['GOODS_NM'] == '꼬막무침 (250g)'] = np.nan
C:\Users\user\AppData\Local\Temp\ipykernel_4060\3931068277.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GOODS_NM'].loc[df['GOODS_NM'] == '고소한도토리묵무침(265g)'] = np.nan
C:\Users\user\AppData\Local\Temp\ipykernel_4060\3931068277.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

Unnamed: 0                       int64
ORD_NO                           int64
H_YMD                   datetime64[ns]
O_YMD                           object
PKG_GOODS_NO                    object
PKG_GOODS_NM                    object
GOODS_NO                        object
GOODS_NM                        object
STD_GSGR_NO_LEV1_NM             object
ORD_QTY                          int64
CANCEL_QTY                       int64
RET_QTY                          int64
REAL_ORD_QTY                     int64
RECVR_ROAD_BASE_ADDR            object
SALE_PRICE                       int64
DISCOUNT_AMT                   float64
FINAL_PRICE                      int64
dtype: object


,ORD_QTY,CANCEL_QTY,RET_QTY,REAL_ORD_QTY,SALE_PRICE,DISCOUNT_AMT,FINAL_PRICE,SALE_PERCETANGE,year,month,...,GOODS_NO_2010014667,GOODS_NO_2010014707,GOODS_NO_2012014895,GOODS_NO_2103015231,GOODS_NO_2204016772,GOODS_NO_3273,GOODS_NO_4092,STD_GSGR_NO_LEV1_NM_국,STD_GSGR_NO_LEV1_NM_메인요리,STD_GSGR_NO_LEV1_NM_반찬
0,0.062844,0.0,0.0,0.062844,0.028541,-0.003333,0.046430,-0.004798,0.062641,-0.001621,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.157143,0.137218,0.000000,0.076012,0.055401
1,0.279654,0.0,0.0,0.279654,0.201421,0.029346,0.242745,0.034917,0.279363,0.020939,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.358571,0.259398,0.000000,0.237149,0.220535
2,0.201362,0.0,0.0,0.201362,0.143214,0.071980,0.163148,0.079625,0.201102,0.012792,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.338346,0.000000,0.210536,0.123850
3,0.269180,0.0,0.0,0.269180,0.236676,0.036113,0.282753,0.037995,0.268894,0.019849,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.008571,0.223684,0.000000,0.338680,0.084278
4,0.101859,0.0,0.0,0.101859,0.047207,-0.004304,0.068759,-0.005407,0.101640,0.009904,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.128571,0.278195,0.000000,0.075647,0.119572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0.596229,0.0,0.0,0.596229,0.482792,0.263050,0.552581,0.316965,0.596577,0.376642,...,0.183459,0.235081,0.325730,0.150741,1.108303,0.562857,0.475564,0.698473,0.270507,0.659465
123,0.637078,0.0,0.0,0.637078,0.498131,0.259862,0.571841,0.326978,0.637450,0.397894,...,0.200000,0.421338,0.268248,0.177100,1.061372,0.588571,0.437970,0.820611,0.259570,0.732193
124,0.600419,0.0,0.0,0.600419,0.487551,0.225170,0.566540,0.272852,0.600769,0.378821,...,0.222556,0.280289,0.253650,0.143328,0.989170,0.641429,0.298872,0.664122,0.244258,0.699037
125,0.594527,0.0,0.0,0.594527,0.475572,0.209203,0.563607,0.249718,0.594874,0.375756,...,0.206015,0.254973,0.306569,0.119440,0.956679,0.541429,0.406015,0.553435,0.242618,0.697540
